In [81]:
import openai
from dotenv import load_dotenv
import os
import requests
from datetime import datetime, time, timezone, timedelta
import random
from bs4 import BeautifulSoup
import json
import re

load_dotenv()
API_URL = os.getenv("API_URL")

In [82]:
class Oracao:

    def __init__(self, day, month, year):
        self.url = f'https://www.vaticannews.va/pt/palavra-do-dia/{year}/{"{:02}".format(month)}/{"{:02}".format(day)}.html'
        self.referenceDate = f'{year}-{"{:02}".format(month)}-{"{:02}".format(day)}T00:00:00.000+00:00'
        self.html = self.get_html()
        
        self.readingRef, self.reading = self.get_reading()
        self.gospelRef, self.gospel = self.get_gospel()
        self.popeWordsRef, self.popeWords = self.get_popewords()



    def get_html(self):
        browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        html_content = requests.get(self.url, headers=browsers)
        soup = BeautifulSoup(html_content.text, 'html.parser')
        return soup
    
    
    
    def corrigir_texto(self, texto):
        # Remover quebras de linha no meio das frases, onde há vírgulas ou dois pontos
        texto_corrigido = re.sub(r'(\,\s*\n|\:\s*\n|\.\s*\n)', lambda m: f'{ m.group(0).strip()}', texto)
        # Juntar as linhas que não estão devidamente separadas (começando com letra minúscula)
        texto_corrigido = re.sub(r'\n([a-z])', r' \1', texto_corrigido)
        # Adicionar quebras de linha nos parágrafos corretos (onde há dois pontos, exclamação ou final de frase com ponto)
        texto_corrigido = re.sub(r'(\.\s+)', r' \1\n\n', texto_corrigido)
        return texto_corrigido.strip()



    def get_reading(self):
        blocks = self.html.find_all(class_='section section--evidence section--isStatic')
        reading = ''
        for b in blocks:
            header = b.find('div', class_='section__head').get_text().strip()
            if header == 'Leitura do Dia':
                reading = b.find('div', class_='section__content').get_text().strip()

        reading_arr = reading.split('\n')
        reading_reference = reading_arr[0].replace('Proclamação do Evangelho de Jesus Cristo segundo ', '').strip()
        reading = '\n'.join(reading_arr[1:]).strip()
        reading = self.corrigir_texto(reading)
        
        return reading_reference, reading



    def get_gospel(self):
        blocks = self.html.find_all(class_='section section--evidence section--isStatic')
        gospel = ''
        for b in blocks:
            header = b.find('div', class_='section__head').get_text().strip()
            if header == 'Evangelho do Dia':
                gospel = b.find('div', class_='section__content').get_text().strip()

        gospel_arr = gospel.split('\n')
        gospel_reference = gospel_arr[0].replace('Proclamação do Evangelho de Jesus Cristo segundo ', '').strip()
        gospel = '\n'.join(gospel_arr[1:]).strip()
        gospel = self.corrigir_texto(gospel)
        
        return gospel_reference, gospel



    def get_popewords(self):
        blocks = self.html.find_all(class_='section section--evidence section--isStatic')
        popeWords = ''
        for b in blocks:
            header = b.find('div', class_='section__head').get_text().strip()
            if header == 'Palavras do Santo Padre':
                popeWords = b.find('div', class_='section__content').get_text().strip()

        popeRefs = re.findall(r'\(([^)]*)\)', popeWords)
        popeRef = ''
        if popeRefs:
            popeRef = popeRefs[-1]
        popeWords = popeWords[:-len(popeRef)-3].strip()
        
        return popeRef, popeWords
    
    
    
    def save_pray(self):
        try:
            url = f'{API_URL}/api/prayer'
            url = f'http://localhost:3000/api/prayer'
            payload = {
                # "uid": "johannesadmin",
                # "supportUid": "johannesadmin",
                "reading": self.reading,
                "readingRef": self.readingRef,
                "gospel": self.gospel,
                "gospelRef": self.gospelRef,
                "popeWords": self.popeWords,
                "popeWordsRef": self.popeWordsRef,
                "referenceDate": self.referenceDate,
                "__t": "daily_pray",
            }
            response = requests.post(url, json=payload)
            if response.status_code != 200:
                print(f'Erro: {response.status_code}')
        except Exception as e:
            print("Problema ao salvar notícia:", e)



In [84]:
# Baseado nas datas disponíveis no calendário de https://www.vaticannews.va/pt/palavra-do-dia.html
year = 2024
month = 9

for day in range(1, 24):
    try:
        o = Oracao(day, month, year)
        o.save_pray()
    except Exception as e:
        print(f'Erro no dia {day}/{month}/{year}:', e)
